In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

# General imports
import os
import cv2
import glob 
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.svm import *
from sklearn.tree import *
from sklearn.metrics import *
from sklearn.ensemble import *
from sklearn.linear_model import *
from sklearn.decomposition import *
from sklearn.preprocessing import *
from sklearn.model_selection import *

SEED = 42
np.random.seed(SEED)
sns.set_style("dark")
mpl.rcParams['figure.dpi'] = 200
%matplotlib inline

**Helper functions**

In [ ]:
def rmse(preds, y):
    mse = mean_squared_error(preds, y)
    return np.sqrt(mse)

def write_subfile(reg, scaler=None, filename="submission.csv"):
    test_data = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")
    ids = test_data["id"]
    test_data = test_data.drop(columns=["id"])
    if scaler:
        test_data = scaler.transform(test_data)
    y = reg.predict(test_data)
    df = pd.DataFrame({
        "id" : ids,
        "loss" : y
    })
    df.to_csv(filename, index=False)

**Reading and scaling data**

In [ ]:
data = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
data.head()

In [ ]:
data.describe()

In [ ]:
Y = data["loss"]
X = data.drop(columns=["id", "loss"])

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X) 

X_scaled.shape, Y.shape

In [ ]:
import optuna
from optuna.samplers import TPESampler

optuna.logging.set_verbosity(optuna.logging.WARNING)

**Tuning XGB Regressor using Optuna**

In [ ]:
from xgboost import XGBRegressor

# Setup XGB hyperparameters for exps
def get_xgb_hyperparams(trail):
    xgb_params = {
        'learning_rate': 0.01,
        'tree_method': 'gpu_hist',
        'booster': 'gbtree',
        'n_estimators': trail.suggest_int('n_estimators', 500, 2500, 100),
        'reg_lambda': trail.suggest_int('reg_lambda', 1, 100),
        'reg_alpha': trail.suggest_int('reg_alpha', 1, 100),
        'subsample': trail.suggest_float('subsample', 0.2, 1.0, step=0.1),
        'colsample_bytree': trail.suggest_float('colsample_bytree', 0.2, 1.0, step=0.1),
        'max_depth': trail.suggest_int('max_depth', 3, 10), 
        'min_child_weight': trail.suggest_int('min_child_weight', 2, 10),
        'gamma': trail.suggest_float('gamma', 0, 20)        
    }
    return xgb_params

In [ ]:
# Define objective function
def objective_xgb(trail, X, Y, n_splits=4):
       
    xgb_params = get_xgb_hyperparams(trail)
    
    kfolds = KFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    total_loss = 0
    
    for train_index, val_index in kfolds.split(X):
        x_train, x_val = X[train_index], X[val_index]
        y_train, y_val = Y[train_index], Y[val_index]
    
        xgb_reg = XGBRegressor(**xgb_params)
    
        xgb_reg = xgb_reg.fit(x_train, y_train)
        preds = xgb_reg.predict(x_val)
    
        total_loss += rmse(preds, y_val)
    
    return total_loss / n_splits

In [ ]:
# Callback function to print log messages when the best trail is updated

def logging_callback(study, frozen_trail):
    prev_best = study.user_attrs.get('prev_best', None)
    if prev_best != study.best_value:
        study.set_user_attr('prev_best', study.best_value)
        print(f"Trail {frozen_trail.number} finished with best value {frozen_trail.value}")




In [ ]:
%%time
study = optuna.create_study(sampler=TPESampler(seed=SEED), 
                            direction='minimize', 
                            study_name='xgb_tuning')
objc = lambda trail : objective_xgb(trail, X_scaled, Y)

study.optimize(objc, n_trials=2, timeout=60*60, callbacks=[logging_callback])

In [ ]:
print(f"Best rmse value: {study.best_value}")
print(f"Best params: ")
for param, value in study.best_params.items():
    print(f"\t{param} : {value}")

In [ ]:
xgb_reg = XGBRegressor(**study.best_params,
                      learning_rate= 0.01,
                      tree_method= 'gpu_hist',
                      booster= 'gbtree')

xgb_reg.fit(X_scaled, Y, eval_metric='rmse', verbose=True)

print(f"RMSE on training data : {rmse(Y, xgb_reg.predict(X_scaled))}")

In [ ]:
write_subfile(xgb_reg, scaler, "xgb_tuned.csv")